# Emojify!

In [1]:
import numpy as np
import csv
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def read_csv(filename):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X, Y

X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/tesss.csv')

In [3]:
maxLen = len(max(X_train, key=len).split())

In [4]:
X_train,Y_train

(array(['never talk to me again', 'I am proud of your achievements',
        'It is the worst day in my life', 'Miss you so much',
        'food is life', 'I love you mum', 'Stop saying bullshit',
        'congratulations on your acceptance', 'The assignment is too long ',
        'I want to go play', 'she did not answer my text ',
        'Your stupidity has no limit', 'how many points did he score',
        'my algorithm performs poorly', 'I got approved',
        'Stop shouting at me', 'Sounds like a fun plan ha ha',
        'no one likes him', 'the game just finished',
        'I will celebrate soon', 'So sad you are not coming',
        'She is my dearest love', 'Good job', 'It was funny lol',
        'candy is life ', 'The chicago cubs won again', 'I am hungry',
        'I am so excited to see you after so long',
        'you did well on you exam', 'lets brunch some day', 'he is so cute',
        'How dare you ask that', 'do you want to join me for dinner ',
        'I said yes',

In [5]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

for idx in range(10):
    print(X_train[idx], emoji.emojize(emoji_dictionary[(Y_train[idx])], use_aliases=True)

never talk to me again 😞
I am proud of your achievements 😄
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😄
The assignment is too long  😞
I want to go play ⚾


In [6]:
Y_oh_train = np.eye(5)[Y_train.reshape(-1)]
Y_oh_test = np.eye(5)[Y_test.reshape(-1)]

In [7]:
idx = 50
print(f"Sentence '{X_train[idx]}' has label index {Y_train[idx]}, which is emoji {label_to_emoji(Y_train[idx])}", )
print(f"Label index {Y_train[idx]} in one-hot encoding format is {Y_oh_train[idx]}")

Sentence 'I missed you' has label index 0, which is emoji ❤️
Label index 0 in one-hot encoding format is [ 1.  0.  0.  0.  0.]


### Implementing Word Vectors


Loading the `word_to_vec_map`, which contains all the vector representations.

In [8]:
with open('glove.6B.50d.txt', 'r') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
        line = line.strip().split()
        curr_word = line[0]
        words.add(curr_word)
        word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
    i = 1
    word_to_index = {}
    index_to_word = {}
    for w in sorted(words):
        word_to_index[w] = i
        index_to_word[i] = w
        i = i + 1


def sentence_to_avg(sentence, word_to_vec_map):    
    words = sentence.lower().split()
    avg = np.array((word_to_vec_map['i'].shape))
    total = 0
    for w in words:
        try:
            total += word_to_vec_map[w]
        except:
            print(w)
    avg = total/len(words)    
    return avg

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def predict(X, Y, W, b, word_to_vec_map):
    """Given X (sentences) and Y (emoji indices), predict emojis and compute the accuracy of your model over the given set."""
    m = X.shape[0]
    pred = np.zeros((m, 1))
    for j in range(m):                       
        words = X[j].lower().split()
        avg = np.zeros((50,))
        for w in words:
            avg += word_to_vec_map[w]
        avg = avg/len(words)
        Z = np.dot(W, avg) + b
        A = softmax(Z)
        pred[j] = np.argmax(A)    
    print("Accuracy: "  + str(np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:])))) 
    return pred

def print_predictions(X, pred):
    print()
    for i in range(X.shape[0]):
        print(X[i], label_to_emoji(int(pred[i])))

In [15]:
def model(X, Y, word_to_vec_map, learning_rate = 0.01, num_iterations = 400):
    m = Y.shape[0]                          
    n_y = 5                                   
    n_h = 50                                 
    
    # Xavier initialization
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    # Convert Y to Y_onehot with n_y classes
    Y_oh = np.eye(n_y)[Y.reshape(-1)]
    
    # Optimization loop
    for t in range(num_iterations): 
        for i in range(m):      
            avg = sentence_to_avg(X[i],word_to_vec_map)
            z = np.dot(W,avg)+b
            a = softmax(z)
            cost = -np.dot(Y_oh[i],np.log(a))
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz
            W = W - learning_rate * dW
            b = b - learning_rate * db
        
        if t % 100 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            pred = predict(X, Y, W, b, word_to_vec_map)

    return pred, W, b

In [28]:
pred, W, b = model(X_train, Y_train, word_to_vec_map)
print(pred)

Epoch: 0 --- cost = 1.95204988128
Accuracy: 0.348484848485
Epoch: 100 --- cost = 0.0797181872601
Accuracy: 0.931818181818
Epoch: 200 --- cost = 0.0445636924368
Accuracy: 0.954545454545
Epoch: 300 --- cost = 0.0343226737879
Accuracy: 0.969696969697
[[ 3.]
 [ 2.]
 [ 3.]
 [ 0.]
 [ 4.]
 [ 0.]
 [ 3.]
 [ 2.]
 [ 3.]
 [ 1.]
 [ 3.]
 [ 3.]
 [ 1.]
 [ 3.]
 [ 2.]
 [ 3.]
 [ 2.]
 [ 3.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 0.]
 [ 2.]
 [ 2.]
 [ 2.]
 [ 1.]
 [ 4.]
 [ 3.]
 [ 3.]
 [ 4.]
 [ 0.]
 [ 3.]
 [ 4.]
 [ 2.]
 [ 0.]
 [ 3.]
 [ 2.]
 [ 2.]
 [ 3.]
 [ 4.]
 [ 2.]
 [ 2.]
 [ 0.]
 [ 2.]
 [ 3.]
 [ 0.]
 [ 3.]
 [ 2.]
 [ 4.]
 [ 3.]
 [ 0.]
 [ 3.]
 [ 3.]
 [ 3.]
 [ 4.]
 [ 2.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 3.]
 [ 4.]
 [ 4.]
 [ 2.]
 [ 2.]
 [ 1.]
 [ 2.]
 [ 0.]
 [ 3.]
 [ 2.]
 [ 2.]
 [ 0.]
 [ 3.]
 [ 3.]
 [ 1.]
 [ 2.]
 [ 1.]
 [ 2.]
 [ 2.]
 [ 4.]
 [ 3.]
 [ 3.]
 [ 2.]
 [ 4.]
 [ 0.]
 [ 0.]
 [ 3.]
 [ 3.]
 [ 3.]
 [ 3.]
 [ 2.]
 [ 0.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 0.]
 [ 2.]
 [ 2.]
 [ 2.]
 [ 3.]
 [ 2.]
 [ 2.]
 [ 

### Examining test set performance 

In [29]:
print("Training set:")
pred_train = predict(X_train, Y_train, W, b, word_to_vec_map)
print('Test set:')
pred_test = predict(X_test, Y_test, W, b, word_to_vec_map)

Training set:
Accuracy: 0.977272727273
Test set:
Accuracy: 0.857142857143


In [30]:
X_my_sentences = np.array(["i adore you", "i love you", "funny lol", "lets play with a ball", "food is ready", "not feeling happy"])
Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])

pred = predict(X_my_sentences, Y_my_labels , W, b, word_to_vec_map)
print_predictions(X_my_sentences, pred)

Accuracy: 0.833333333333

i adore you ❤️
i love you ❤️
funny lol 😄
lets play with a ball ⚾
food is ready 🍴
not feeling happy 😄


## Using LSTMs in Keras: 

Let's build an LSTM model that takes word sequences as input!

In [32]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

Using TensorFlow backend.


In [52]:
def sentences_to_indices(X, word_to_index, max_len):
    m = X.shape[0]                                   # number of training examples
    X_indices = np.zeros((m,max_len))
    for i in range(m):                               # loop over training examples
        sentence_words = X[i].lower().split()
        j = 0
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w]           
            j = j+1    
    return X_indices

In [65]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1                  
    emb_dim = word_to_vec_map["cucumber"].shape[0]      
    emb_matrix = np.zeros((vocab_len,emb_dim))
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]
    embedding_layer = Embedding(vocab_len,emb_dim,trainable=False) 
    embedding_layer.build((None,)) 
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

In [70]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape,dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)
    X = LSTM(128,return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128)(X)
    X = Dropout(0.5)(X)
    X = Dense(5)(X)
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices,outputs=X)
    return model

In [71]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
__________

In [72]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [73]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = np.eye(5)[Y_train.reshape(-1)]

In [74]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
132/132 [==============================] - 0s - loss: 1.6033 - acc: 0.2576     
Epoch 2/50
132/132 [==============================] - 0s - loss: 1.5318 - acc: 0.3712     
Epoch 3/50
132/132 [==============================] - 0s - loss: 1.4882 - acc: 0.3409     
Epoch 4/50
132/132 [==============================] - 0s - loss: 1.4506 - acc: 0.4167     
Epoch 5/50
132/132 [==============================] - 0s - loss: 1.3445 - acc: 0.4924     
Epoch 6/50
132/132 [==============================] - 0s - loss: 1.2244 - acc: 0.5379     
Epoch 7/50
132/132 [==============================] - 0s - loss: 1.1206 - acc: 0.5833     
Epoch 8/50
132/132 [==============================] - 0s - loss: 1.0129 - acc: 0.5682     
Epoch 9/50
132/132 [==============================] - 0s - loss: 0.9789 - acc: 0.6288     
Epoch 10/50
132/132 [==============================] - 0s - loss: 0.7978 - acc: 0.7045     
Epoch 11/50
132/132 [==============================] - 0s - loss: 0.7119 - acc: 0.7576   

In [75]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print("Test accuracy = ", acc)

132/132[==============================] -- ETA: 0s
Test accuracy =  0.93.2571437086
